In [36]:
import tensorflow
import pandas as pd
import numpy as np

from skimage.feature import hog
from skimage.io import imread, imshow

from keras_core import Sequential
from keras_core.models import Model
from keras_core.layers import Dense, InputLayer, Conv2D, MaxPooling2D, Dropout, Flatten, LeakyReLU, BatchNormalization, Add

In [2]:
# Reading
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
df_val = pd.read_csv('./data/val.csv')

In [3]:
df_train['Data'] = df_train['Image'].apply(lambda x: imread(f'./data/train_images/{x}', as_gray=True))
df_test['Data'] = df_test['Image'].apply(lambda x: imread(f'./data/test_images/{x}', as_gray=True))
df_val['Data'] = df_val['Image'].apply(lambda x: imread(f'./data/val_images/{x}', as_gray=True))

In [52]:
# HOG Features
df_train['Features'] = df_train['Data'].apply(lambda x: hog(x, orientations=9, pixels_per_cell=(4, 4), cells_per_block=(2, 2), channel_axis=-1))
df_val['Features'] = df_val['Data'].apply(lambda x: hog(x, orientations=9, pixels_per_cell=(4, 4), cells_per_block=(2, 2), channel_axis=-1))
df_test['Features'] = df_test['Data'].apply(lambda x: hog(x, orientations=9, pixels_per_cell=(4, 4), cells_per_block=(2, 2), channel_axis=-1))

In [53]:
clf = Sequential([
    InputLayer(shape=(8100, )),
    Dense(2000, activation='relu'),
    Dense(2000, activation='relu'),
    Dense(100, activation='softmax')
]) 

In [8]:
df_train['DataNorm'] = df_train['Data'].apply(lambda x: x - x.mean())
df_val['DataNorm'] = df_val['Data'].apply(lambda x: x - x.mean())
df_test['DataNorm'] = df_test['Data'].apply(lambda x: x - x.mean())

In [10]:
# Reshape data
X_train = np.stack(df_train['DataNorm']).reshape(-1, 64, 64, 1)
X_val = np.stack(df_val['DataNorm']).reshape(-1, 64, 64, 1)

y_train = df_train['Class'].to_numpy()
y_val = df_val['Class'].to_numpy()

In [ ]:
def ConvBatch(in_num, out_num, kernel_size=3, padding='same', stride=1):
    return [
        Conv2D(out_num, kernel_size=kernel_size, strides=stride, padding=padding, use_bias=False),
        BatchNormalization(),
        LeakyReLU()
    ]

In [ ]:
def ResidualBlock(in_channels):
    reduced_channels = in_channels // 2

    input_tensor = Input(shape=(None, None, in_channels))
    x = ConvBatch(in_channels, reduced_channels, kernel_size=1, padding='valid')(input_tensor)
    x = ConvBatch(reduced_channels, in_channels)(x)
    output_tensor = Add()([x, input_tensor])
    
    return Model(inputs=input_tensor, outputs=output_tensor)

In [31]:
def ResidualBlock(in_channels):
    reduced_channels = in_channels // 2

    rb1 = Conv2D(filters=in_channels, kernel_size=1, padding='same')
    rb1 = BatchNormalization(reduced_channels)(rb1)
    rb1 = LeakyReLU(rb1)

    rb2 = Conv2D(filters=reduced_channels, kernel_size=3, padding='same')
    rb2 = BatchNormalization(in_channels)(rb1)
    rb2 = LeakyReLU(rb1)

    out = rb1
    out = rb2(out)
    out = Add([out, out])
    return out

In [32]:
ResidualBlock(10)

ValueError: Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: <Conv2D name=conv2d_5, built=False> (of type <class 'keras_core.src.layers.convolutional.conv2d.Conv2D'>)

In [24]:
def MakeResidualBlock(in_channels, blocks):
    return Sequential(*[ResidualBlock(in_channels) for _ in range(blocks)])

In [25]:
MakeResidualBlock(64, 1)

TypeError: ResidualBlock() missing 2 required positional arguments: 'kernel_size' and 'filters'

In [11]:
clf = Sequential([
    InputLayer(shape=(64, 64, 1)),
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(32),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(64),
    MakeResidualBlock(in_channels=64, blocks=1),
    Conv2D(128, (3, 3), activation='relu'),
    MakeResidualBlock(in_channels=64, blocks=1),
    Conv2D(128, (3, 3), activation='relu'),
    MakeResidualBlock(in_channels=64, blocks=1),
    Conv2D(128, (3, 3), activation='relu'),
    MakeResidualBlock(in_channels=64, blocks=1),
    Flatten(),
    Dense(1000, activation='relu'),
    Dropout(0.3),
    Dense(100, activation='softmax')
])

In [12]:
clf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
clf.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), verbose=1, batch_size=64)

Epoch 1/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 56s 256ms/step - accuracy: 0.0312 - loss: 4.4826 - val_accuracy: 0.1815 - val_loss: 3.2577
Epoch 2/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 52s 252ms/step - accuracy: 0.2736 - loss: 2.8694 - val_accuracy: 0.2870 - val_loss: 2.7898
Epoch 3/20
 29/204 ━━━━━━━━━━━━━━━━━━━━ 42s 243ms/step - accuracy: 0.4810 - loss: 1.9565

KeyboardInterrupt: 